# Логистическая регрессия

Обучим логистическую регрессию для предсказания того, откликнется клиент  
на рекламное предложение (target = 1) или нет (target = 0).

## Подлючение библиотек, загрузка и обзор данных

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [5]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

**Задание**  
Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?  
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,  
имея в виду 41 процент).

In [9]:
y[y==0].count()/y.shape[0]

0.8809695854956316

Обучим логистическую регрессию с параметрами по умолчанию.

In [10]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [11]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [13]:
from sklearn.metrics import f1_score

f1_score(prediction,ytest)

0.0

**Вопрос**  
Чему равен `f1_score`?

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [14]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]

array([[0.86331972, 0.13668028],
       [0.86305957, 0.13694043],
       [0.80967732, 0.19032268],
       [0.73110167, 0.26889833],
       [0.87780414, 0.12219586],
       [0.96122864, 0.03877136],
       [0.95965491, 0.04034509],
       [0.96560659, 0.03439341],
       [0.92114871, 0.07885129],
       [0.9349701 , 0.0650299 ]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос**  
Чему равен `f1_score`, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [16]:
new_predict=(probs_test[:,1]>0.1).astype(int)
f1_score(new_predict,ytest)

0.2469521725539231

У обученной модели можно посмотреть веса (как и в линейной регрессии).  
Выведем на экран веса модели (`model.coef_`, `model.intercept_`).

In [17]:
lr.coef_, lr.intercept_

(array([[-5.55214705e-02, -7.90868782e-04, -4.28172545e-04,
         -1.07248154e-03, -1.34958177e-03, -4.95807875e-04,
          1.13131722e-05, -1.80011616e-03, -1.26843809e-03,
         -1.45283567e-05]]),
 array([-0.00115086]))

Создайте `pd.DataFrame`, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).  
Отсортируйте таблицу по убыванию весов.

**Вопрос**  
Какой признак имеет наибольший положительный вес?

In [27]:
coef=lr.coef_[0]

array([-5.55214705e-02, -7.90868782e-04, -4.28172545e-04, -1.07248154e-03,
       -1.34958177e-03, -4.95807875e-04,  1.13131722e-05, -1.80011616e-03,
       -1.26843809e-03, -1.45283567e-05])

In [44]:
dic=dict()
coef=lr.coef_[0]
for a,b in zip(Xtrain.columns,coef):
    dic[a]=[b]

df=pd.DataFrame(dic)
df

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM
0,-0.055521,-0.000791,-0.000428,-0.001072,-0.00135,-0.000496,0.000011,-0.0018,-0.001268,-0.000015


## Бонус

**Задание 1**  
Подберите порог для перевода вероятностей в классы, дающий максимальное значение `f1_score`.

In [50]:
porog=np.linspace(0.05,0.3,10)
min=dict()
for i in porog:
     new_predict=(probs_test[:,1]>i).astype(int)
     f_1=f1_score(new_predict,ytest)
     min[i]=f_1
min

{0.05: 0.23141967067200714,
 0.07777777777777778: 0.23915401301518435,
 0.10555555555555556: 0.24545454545454548,
 0.13333333333333333: 0.24355414518048396,
 0.1611111111111111: 0.2314300151591713,
 0.18888888888888888: 0.22395833333333331,
 0.21666666666666667: 0.183803457688808,
 0.2444444444444444: 0.09166666666666667,
 0.2722222222222222: 0.01748251748251748,
 0.3: 0.01061946902654867}

**Задание 2**  
Во вложенном цикле подберите одновременно коэффициент регуляризации `C` у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение `f1_score`.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: `Xtrain`, `Xval`, `Xtest`.

*   В цикле при подборе `С` и порога будем обучаться по `Xtrain`, а предсказывать и измерять качество по `Xval`.

*   Качество итоговой модели с найденными `C` и порогом измерьте по `Xtest`.

Так не переобучимся!


In [52]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

alpha=np.logspace(-2,3,25)
porog=np.linspace(0.03,0.2,10)

qual=list()

for a in alpha:
    for i in porog:
        model = LogisticRegression(C = a)
        model.fit(Xtrain, ytrain)
        prediction = model.predict_proba(Xval)

        
        new_predict=(prediction[:,1]>i).astype(int)
        f_1=f1_score(new_predict,yval)

        
        qual.append([a,i,f_1])
        

In [60]:
qual=np.array(qual)
min_index = np.argmax(qual[:, 2])

# Получаем строку с минимальным значением в третьем столбце
min_value_row = qual[min_index]
min_value_row 

array([0.01      , 0.10555556, 0.2278481 ])

In [61]:
model = LogisticRegression(C = 0.01 )

model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных

prediction = model.predict_proba(Xtest)
new_predict=(prediction[:,1]>0.105).astype(int)

f1_score(new_predict,ytest)


0.24481865284974091